Load and pre-process ACIC data set.

In [1]:
### LINES TO IMPORT THE DECONFOUNDER PACKAGE IN THE PARENT FOLDER ###
import os
import sys
sys.path.append("..")
### IMPORTS
from deconfounder.causal_tree import CausalTree
from deconfounder.deconfounder_tree import DeconfounderTree
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import polyfit
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import time

def fit_causal_tree(X, y):
    # Fit and tune causal tree
    start_time = time.time()
    tuned_parameters = [{'min_samples_leaf': range(10, 300, 10)}]
    causal_tree = GridSearchCV(CausalTree(random_state=42), tuned_parameters, cv=5)
    causal_tree.fit(X, y)   
    #print("Best parameters set found on development set:")
    print(causal_tree.best_params_)
    #print("--- Time to fit (and tune) causal tree %s seconds ---" % (time.time() - start_time))
    return causal_tree

def evaluate(preds, effs):
    mse = np.mean((preds - effs)**2)
    avg_effect = round(np.mean(effs), 2)
    avg_with_policy = round(np.mean((preds > 0) * effs), 2)
    improvement = round(100*avg_with_policy/avg_effect-100, 2)
    return mse, improvement

##### LOAD FEATURE DATA
df_X = pd.read_csv("../data/x.csv")
df_X = pd.get_dummies(df_X)
all_feature_names = df_X.columns

##### GET RESPONSE FILES
response_files = [f for f in listdir("../data/77") if isfile(join("../data/77", f))]

mse_rows = []
improvement_rows = []
#### CONDUCT ANALYSIS FOR EACH FILE
for f_name in response_files:
    np.random.seed(42)
    print(f_name)
    ##### LOAD RESPONSE DATA
    # Merge with response info
    df = pd.concat([df_X, pd.read_csv(f"../data/77/{f_name}")], axis=1)
    # Create observed response and assignment variables
    df.rename(columns={"z": "z_obs"}, inplace=True)
    df['effs'] = df.mu1 - df.mu0
    df['y_obs'] = df.y1 * df.z_obs + df.y0 * (1 - df.z_obs)
    df['z_exp'] = np.random.binomial(1, df.z_obs.mean(), df.shape[0])
    df['y_exp'] = df.y1 * df.z_exp + df.y0 * (1 - df.z_exp)

    ##### INTRODUCE CONFOUNDING 
    n_confounders = 5 
    corr_matrix = df[all_feature_names.values.tolist() + ['z_obs', 'y_obs']].corr()
    corr_matrix
    ranked_features = (corr_matrix.z_obs.abs() * corr_matrix.y_obs.abs()).sort_values(ascending=False).index.values
    confounders = ranked_features[~np.isin(ranked_features, ['z_obs', 'y_obs'])][:n_confounders]
    feature_names = all_feature_names[~np.isin(all_feature_names, confounders)]
    X = df[feature_names].copy()

    ##### SPLIT INTO TRAIN AND TEST 
    is_train = np.full(df.shape[0], True)
    is_train[:802] = False
    np.random.shuffle(is_train)

    ##### FIT AND EVALUATE OBSERVATIONAL AND EXPERIMENTAL TREES
    trees = []
    mse_all = []
    improvement_all = []
    exp_size = 1000
    for d_type in ["OBSERVATIONAL", "EXPERIMENTAL"]:
        if d_type == "OBSERVATIONAL":
            size = 4000
            y = df.y_obs
            z = df.z_obs
        else:
            size = exp_size
            y = df.y_exp
            z = df.z_exp
        print(f"Fit causal tree with an {d_type} data set of {size} individuals")
        X_with_treatment = X.copy()
        X_with_treatment['treated'] = z
        causal_tree = fit_causal_tree(X_with_treatment[is_train][:size], y[is_train][:size])
        trees.append(causal_tree)
        preds = causal_tree.predict(X_with_treatment[~is_train])
        mse, improvement = evaluate(preds, df.effs[~is_train])
        mse_all.append(mse)
        improvement_all.append(improvement)

    ##### MERGE OBSERVATIONAL AND EXPERIMENTAL DATA
    X_all = pd.concat([X[is_train], X[is_train][:exp_size]], ignore_index=True)
    X_all['experiment'] = False
    X_all.loc[4000:, 'experiment'] = True
    X_all['treated'] = False
    X_all.loc[~X_all.experiment.values, 'treated'] = df[is_train].z_obs.values
    X_all.loc[X_all.experiment.values, 'treated'] = df[is_train][:exp_size].z_exp.values
    y_all = pd.concat([df[is_train].y_obs, df[is_train][:exp_size].y_exp], ignore_index=True)
    X_all = X_all.sample(frac=1)
    y_all = y_all.loc[X_all.index.values]

    ##### BUILD DECONFOUNDER TREE
    print("Fit deconfounder tree")
    tuned_parameters = [{'min_weight_fraction_leaf': np.array(range(5, 55, 5))/100}]
    deconfounder = GridSearchCV(DeconfounderTree(random_state=42), tuned_parameters, cv=5)
    deconfounder.fit(X_all, y_all)
    print(deconfounder.best_params_)
    pd.Series(deconfounder.predict(X_all)).value_counts()

    ##### TEST DECONFOUNDED MODEL
    X_test = X[~is_train]
    bias_preds = deconfounder.predict(X_test)
    obs_preds = trees[0].predict(X_test)
    preds = obs_preds - bias_preds
    mse, improvement = evaluate(preds, df.effs[~is_train])
    mse_all.append(mse)
    improvement_all.append(improvement)
    print(f"MSE: {mse_all}")
    print(f"IMPROVEMENT: {improvement_all}")
    mse_rows.append(mse_all)
    improvement_rows.append(improvement_all)
    print("-------")

zymu_129047995.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 160}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 150}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.35}
MSE: [8.97567934419504, 7.746484983793969, 6.784245182459641]
IMPROVEMENT: [4.4, -0.73, 4.89]
-------
zymu_129047999.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 120}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 250}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.45}
MSE: [34.61506060168141, 44.71371100971731, 38.62145136801542]
IMPROVEMENT: [1.01, 0.0, -1.01]
-------
zymu_129048005.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 130}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 270}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.45}
MSE: [81.137

{'min_weight_fraction_leaf': 0.5}
MSE: [25.053045556094908, 25.203966666704776, 24.66548435295143]
IMPROVEMENT: [117.24, 146.55, 146.55]
-------
zymu_129048098.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 120}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 100}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.2}
MSE: [17.89615914826332, 23.639092424748178, 22.63700044734615]
IMPROVEMENT: [94.92, 94.35, 78.53]
-------
zymu_129048099.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 290}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 50}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.5}
MSE: [20.44999477216869, 24.19989619501063, 20.355677232984892]
IMPROVEMENT: [0.0, 0.0, 0.0]
-------
zymu_129048104.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 280}
Fit causal tree 

{'min_samples_leaf': 170}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 40}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.5}
MSE: [36.09498636262932, 40.8697713126524, 33.75834513662394]
IMPROVEMENT: [4.26, -0.67, 4.26]
-------
zymu_129048223.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 220}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 290}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.5}
MSE: [22.119780529926903, 18.753429831574127, 20.017232766319445]
IMPROVEMENT: [0.0, 0.0, 0.0]
-------
zymu_129048226.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 110}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 280}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.45}
MSE: [19.310649352498334, 19.32551198080188, 19.401470779143256]
IMPROVEMENT: [-1400.0, -625.0, -120

{'min_samples_leaf': 190}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 150}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.5}
MSE: [15.584171183494382, 15.17381609395495, 15.495677868606444]
IMPROVEMENT: [12.74, 10.51, 12.1]
-------
zymu_129048278.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 270}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 240}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.5}
MSE: [19.941655355824576, 23.936354045489438, 18.696934948747774]
IMPROVEMENT: [0.0, 0.0, 0.0]
-------
zymu_129048279.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 290}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 260}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.4}
MSE: [41.94125353032262, 40.150447404535385, 40.25841401809807]
IMPROVEMENT: [0.0, 0.0, 0.0]
---

{'min_samples_leaf': 170}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 70}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.45}
MSE: [21.855760397304213, 18.19140947213511, 15.264839814956373]
IMPROVEMENT: [0.0, -3.2, -0.84]
-------
zymu_129048364.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 190}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 170}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.5}
MSE: [17.53886778579917, 16.116486157170606, 14.325241964101675]
IMPROVEMENT: [0.0, 0.0, 0.0]
-------
zymu_129048366.csv
Fit causal tree with an OBSERVATIONAL data set of 4000 individuals
{'min_samples_leaf': 270}
Fit causal tree with an EXPERIMENTAL data set of 1000 individuals
{'min_samples_leaf': 240}
Fit deconfounder tree
{'min_weight_fraction_leaf': 0.45}
MSE: [9.329893613883867, 4.775965008888235, 5.247826441270494]
IMPROVEMENT: [0.0, 0.0, 0.0]
------

In [2]:
mse_cols = ["MSE_Obs", "MSE_Exp", "MSE_Comb"]
imp_cols = ["Imp_Obs", "Imp_Exp", "Imp_Comb"]
results = pd.concat([pd.DataFrame(mse_rows, columns=["MSE_Obs", "MSE_Exp", "MSE_Comb"]),
                     pd.DataFrame(improvement_rows, columns=["Imp_Obs", "Imp_Exp", "Imp_Comb"])], axis=1)
print("EFFECT ESTIMATION")
print("Number of times it hurts observational estimations:", (results["MSE_Comb"] > results["MSE_Obs"]).sum())
print("Number of times it is irrelevant for observational estimations:", (results["MSE_Comb"] == results["MSE_Obs"]).sum())
print("Number of times it helps observational estimations:", (results["MSE_Comb"] < results["MSE_Obs"]).sum())
print("Number of times it makes the best estimations", (results[mse_cols].idxmin(axis=1) == "MSE_Comb").sum())
print("DECISION MAKING")
print("Number of times it hurts observational decisions:", (results["Imp_Comb"] < results["Imp_Obs"]).sum())
print("Number of times it is irrelevant for observational decisions:", (results["Imp_Comb"] == results["Imp_Obs"]).sum())
print("Number of times it helps observational decisions:", (results["Imp_Comb"] > results["Imp_Obs"]).sum())
print("Number of times it makes the best decisions", (results[imp_cols].idxmax(axis=1) == "Imp_Comb").sum())

EFFECT ESTIMATION
Number of times it hurts observational estimations: 35
Number of times it is irrelevant for observational estimations: 0
Number of times it helps observational estimations: 65
Number of times it makes the best estimations 48
DECISION MAKING
Number of times it hurts observational decisions: 44
Number of times it is irrelevant for observational decisions: 36
Number of times it helps observational decisions: 20
Number of times it makes the best decisions 11


In [5]:
results.to_csv("../data/results.csv", index=False)

In [6]:
import pandas as pd

df = pd.read_csv("../data/results.csv")
df

,MSE_Obs,MSE_Exp,MSE_Comb,Imp_Obs,Imp_Exp,Imp_Comb
0,8.975679,7.746485,6.784245,4.40,-0.73,4.89
1,34.615061,44.713711,38.621451,1.01,0.00,-1.01
2,81.137482,88.783700,88.164459,0.00,0.00,0.00
3,37.691499,38.883017,38.428639,0.00,0.00,-7.61
4,13.496790,12.717424,15.394683,-8.55,0.00,-9.29
5,31.815242,58.737379,38.309421,24.30,-0.35,17.96
6,27.177058,28.113881,26.305743,3.54,0.00,10.62
7,38.198783,34.554015,41.672704,0.00,0.00,-5.57
8,57.040287,66.581846,56.665855,3.47,-3.10,3.47
9,6.609392,3.682332,5.242631,-10.80,0.00,0.35
